# Projets Retrieval-Augmented Generation (RAG)
- Chatbot pour FAQ pour une entreprise de gestion des fonds immobiliers

## Installation des modules

In [ ]:
# Créer un environnement virtuel nommé myenv et installer les librairies
! python -m venv myenv
! .\myenv\Scripts\activate
! python.exe -m pip install --upgrade pip
! pip install requirements.txt
! pip list 

## Chargement du LLM et définition de la clé OPENAI

In [166]:
import os
from langchain.chat_models import ChatOpenAI


with open("OpenAIkey.txt", "r") as file:
    mykey = file.read().strip()  

os.environ["OPENAI_API_KEY"] = mykey


### 1. Chargement des documents d'information sur l'entreprise

In [ ]:
from langchain.document_loaders import PyPDFLoader

# Création d'une fonction pour charger les pdfs dans une liste

def pdfloader(folder:str) -> list:

    """--Docstring--
    
    Args: 
        folder: string object (files directory path)
    
    function return a list
    """
    documents = []
    
    for doc in os.listdir(folder):
        file_path = os.path.join(folder, doc)
        file = PyPDFLoader(file_path).load()
        documents.extend(file)
    
    print(f"Nombre Total de pages: {len(documents)}")
    return documents


In [ ]:
# Chargement des documents
Documents = pdfloader("Docs/")

In [ ]:
# Exemple Affichage des informations du premier document
metadata = Documents[2].metadata
content = Documents[2].page_content[:502]
print(f"Metadata:\n\n{metadata}\n\nContent:\n\n{content}")

### 2. Fractionnement des documents (Document Splitting)

In [110]:
from langchain.text_splitter import TokenTextSplitter as tksplit

# Extraction des textes:
Textes = ' '.join([page.page_content for page in Documents])

splitter = tksplit(
                    chunk_size = 500,
                    chunk_overlap = 10 
                  )

# Appliquer le fractionnement
DocTokens = splitter.split_text(Textes)

In [111]:
len(DocTokens)
DocTokens[23]

' la limite de l’autorisation \ndonnée par l’Assemblée Générale. L’Assemblée Générale du 6 juin 2024 a fixé \nà 1 400 M€ le montant maximal de l’endettement financier autorisé.\nL’objectif de gestion est de proposer à l’investisseur une distribution potentielle \nde dividendes ainsi qu’une progression de la valeur des parts souscrites sur \nun horizon de détention long terme. Cet objectif est non garanti. Le taux de \ndistribution potentiel de la SCPI (rapport entre le dividende brut d’une année n \n(y compris les acomptes et la quote-part de plus-values distribuées), avant \nles prélèvements obligatoires et autre fiscalité, et le prix de souscription au  \n1er janvier de cette même année) pourra évoluer sensiblement à la hausse \ncomme à la baisse selon l’évolution de la conjoncture immobilière.\nINVESTISSEURS\xa0DE\xa0DÉTAIL\xa0 VISÉS\nTout investisseur (hors « US Person ») non professionnels, professionnels ou \ncontreparties éligibles souhaitant investir indirectement une partie de

### 3. Vectorstores and Embeddings

In [141]:
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings

openAIembedding = OpenAIEmbeddings(model="text-embedding-ada-002")

vectorstore = Qdrant.from_documents(
                                    Documents,  
                                    openAIembedding,
                                    location=":memory:", 
                                    collection_name="Vectordb",  
                                   )

print("Vectorstore Qdrant created")

Vectorstore Qdrant created


### 4. Créer le modèle RAG (Retrieval-Augmented Generation)

In [148]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

gpt4o = ChatOpenAI(model_name="gpt-4o", temperature=0)

# Créer un retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})

# Créer la chaîne RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
                                        llm=gpt4o,
                                        chain_type="stuff",
                                        retriever=retriever,
                                        return_source_documents=True
                                      )

In [173]:
# Interroger le modèle
query = "Quelle est la rentabilité de Sofiprime?"
result = qa_chain(query)
print(result["result"])
print(result["source_documents"])

La rentabilité de SOFIPRIME dépend des scénarios de performance futurs du marché immobilier. Voici quelques exemples de rendements annuels moyens après déduction des coûts, basés sur différents scénarios :

- Scénario défavorable : 
  - 1 an : -16,34 %
  - 5 ans : -8,98 %
  - 10 ans : -8,02 %

- Scénario intermédiaire :
  - 1 an : -7,06 %
  - 5 ans : 1,11 %
  - 10 ans : 2,18 %

- Scénario favorable :
  - 1 an : -4,73 %
  - 5 ans : 3,65 %
  - 10 ans : 4,74 %

Ces chiffres montrent que la rentabilité peut varier considérablement en fonction des conditions du marché. Il est important de noter que ces rendements ne sont pas garantis et que l'investissement comporte des risques, y compris la possibilité de perte en capital.
[Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 20.1 (Macintosh)', 'creationdate': '2025-03-04T19:17:37+01:00', 'moddate': '2025-03-04T19:17:39+01:00', 'trapped': '/False', 'source': 'Docs/Sofiprime.pdf', 'total_pages': 2, 'page': 0, 